# Toxic Spans Detection

In this notebook, we will train a model to detect toxic spans in text.

We will use [simpletransformers](https://simpletransformers.ai/) that is a wrapper for many popular models available in [Hugging Face](https://huggingface.co/).

We will use a pre-trained model ([neuralmind/bert-base-portuguese-cased · Hugging Face](https://huggingface.co/neuralmind/bert-base-portuguese-cased)) that is trained on Portuguese.

## Imports

In the first cell, we set the `KAGGLE_USERNAME` and `KAGGLE_KEY` environment variables. We also import the required packages.

In [1]:
import os

os.environ["KAGGLE_USERNAME"] = "dougtrajano"
os.environ["KAGGLE_KEY"] = "ce5588a6577391214c6ef22fcb5bd507"

import shutil
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from typing import List, Dict, Any
from kaggle.api.kaggle_api_extended import KaggleApi
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from simpletransformers.question_answering import (
    QuestionAnsweringModel,
    QuestionAnsweringArgs
)

%matplotlib inline

logging.basicConfig(level=logging.INFO)

_logger = logging.getLogger("transformers")
_logger.setLevel(logging.WARNING)

seed = 1993

In the next cell, we will remove some folders used by `simpletransformers`.

In [ ]:
temp_folders = ["cache_dir", "outputs", "runs"]

for folder in temp_folders:
    if os.path.exists(folder):
        shutil.rmtree(folder, ignore_errors=True)

## Functions

In this section, we will define some helper functions.

In [ ]:
def get_toxic_substrings(text: str, spans: List[int], verbose=False) -> List[str]:
    """
    Extract string words based on a list of spans.

    Args:
    - text: The text to extract words from.
    - spans: A list of spans to extract words from.

    Returns:
    - A list of words extracted from the text.
    """
    def format_substring(substring: str):
        return " ".join("".join(substring).split())

    delimiter = None
    words = []
    tmp = []
    for i in range(len(text)):
        if i in spans:
            if verbose:
                print(f"Found span at {i} ({text[i]})")
            if delimiter is None:
                delimiter = i
            else:
                delimiter += 1
            tmp.append(text[i])
        else:
            tmp.append(" ")

        if delimiter is not None and i != delimiter:
            words.append(format_substring(tmp))
            tmp = []
            delimiter = None

    if len(tmp) > 0:
        words.append(format_substring(tmp))
        
    words = [w for w in words if w not in [" ", ""]]
    return words

class DataPreprocessing(object):
    """
    Data preprocessing class.
    """

    def __init__(self):
        self.id = 0

    def __call__(self, X: List[str], Y: List[List[int]]):
        data = []
        for x, y in zip(X, Y):
            row = {"context": x, "qas": []}

            if y is not None:
                y = get_toxic_substrings(x, y)

                for i in y:
                    tmp = {
                        "id": self.id,
                        "is_impossible": False,
                        "question": "What's the toxic substring?",
                        "answers": [
                            {
                                "text": i,
                                "answer_start": x.find(i)
                            }
                        ]
                    }

                    row["qas"].append(tmp)
            else:
                tmp = {
                    "id": self.id,
                    "is_impossible": True,
                    "question": "What's the toxic substring?",
                    "answers": []
                    }

                row["qas"].append(tmp)

            self.id += 1
            data.append(row)
        return data

## Load the data

In this section, we will download the data and load it into a pandas dataframe.

In [ ]:
if not os.path.exists("olidbr.csv"):
    print("Downloading data from Kaggle")
    kaggle = KaggleApi()
    kaggle.authenticate()
    kaggle.dataset_download_file(dataset="olidbr", file_name="olidbr.csv")

df = pd.read_csv("olidbr.csv")

print(f"Shape: {df.shape}")
df.head()

In [ ]:
print(f"Comments with spans assigned: {df[df.toxic_spans.notnull()].shape[0]} ({df[df.toxic_spans.notnull()].shape[0] / df.shape[0] * 100:.2f}%)")
print(f"Comments without spans assigned: {df[df.toxic_spans.isnull()].shape[0]} ({df[df.toxic_spans.isnull()].shape[0] / df.shape[0] * 100:.2f}%)")

We need to filter out the comments that do not have a toxic span.

In [ ]:
df = df[df["toxic_spans"].notnull()]
df.reset_index(drop=True, inplace=True)

df["toxic_spans"] = df["toxic_spans"].apply(lambda x: eval(x))

print(f"Shape: {df.shape}")

## Explorative analysis

In the second cell, we load the data and perform an exploratory analysis.

In [ ]:
toxic_substrs = []

for row in df.to_dict(orient="records"):
    if row.get("toxic_spans") is not None:
        toxic_substrs.extend(get_toxic_substrings(row["text"], row["toxic_spans"]))

print(f"toxic_substrs: {len(toxic_substrs)}")

wc = WordCloud(width=1920, height=1024,
               max_words=200, max_font_size=100)

wc.generate(" ".join(toxic_substrs))

plt.figure(figsize=(15, 10))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

## Prepare the data

In this section, we will prepare the data in order to train the model.

The `simpletransformers` library expects the data in a specific format.

More information about the format can be found in the [Question Answering Data Formats - Simple Transformers](https://simpletransformers.ai/docs/qa-data-formats/)

In [ ]:
df = df[["text", "toxic_spans"]]

X = df["text"].values
y = df["toxic_spans"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=seed)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

In [ ]:
prc = DataPreprocessing()
train_data = prc(X_train, y_train)
eval_data = prc(X_test, y_test)

print("Train data", train_data[0], sep="\n")
print("Eval data", eval_data[0], sep="\n")

## Training the model

In this section, we will train a baseline model to predict if a toxic comment is targeted or not.

We will not perform hyperparameter tuning because it is a simple baseline model.

In [10]:
model_args = QuestionAnsweringArgs(
    num_train_epochs=6,
    evaluate_during_training=True
)

# Create a ClassificationModel
model = QuestionAnsweringModel(
    model_type="bert",
    model_name="neuralmind/bert-base-portuguese-cased",
    args=model_args,
    use_cuda=False
)

# Train the model
model.train_model(train_data, eval_data=eval_data)

## Evaluating the model

In this section, we will evaluate the model with the following metrics:

- **Accuracy**: the percentage of correct predictions;
- **Precision**: the percentage of predicted targeted comments that are actually targeted;
- **Recall**: the percentage of targeted comments that are actually predicted as targeted;
- **F1-Score**: the harmonic mean of precision and recall;
- **ROC AUC**: the area under the receiver operating characteristic Curve (ROC AUC).

In [ ]:
result, texts = model.eval_model(eval_data)

In [ ]:
print(classification_report(y_true, y_pred, digits=4,
                            target_names=classes.values()))

## Testing the model

In the last section, we will test the model with some comments from the test set.

In [ ]:
answers, probabilities = model.predict(eval_data)

print(answers)
print(probabilities)